In [1]:
%pip install -qq "fastrepl[rag]>=0.0.16"
%pip install -qq -U embedchain

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%env OPENAI_API_KEY=

In [2]:
from IPython.display import Markdown

In [3]:
from embedchain import App

pg_bot = App()
pg_bot.add("http://paulgraham.com/greatwork.html")

Doc content has not changed. Skipping creating chunks and embeddings


'161fcad59ba37a28e60a45f5355dab43'

In [4]:
res = pg_bot.query("how to do great work?", dry_run=True)
Markdown(res)


  Use the following pieces of context to answer the query at the end.
  If you don't know the answer, just say that you don't know, don't try to make up an answer.

  interesting, which you necessarily will if you're on the right path, the work will probably feel less burdensome than a lot of your peers'.The discoveries are out there, waiting to be made. Why not by you? Notes[1] I don't think you could give a precise definition of what counts as great work. Doing great work means doing something important so well that you expand people's ideas of what's possible. But there's no threshold for importance. It's a matter of degree, and often hard to judge at the

  Query: how to do great work?

  Helpful Answer:


In [15]:
res = pg_bot.retrieve_from_database("how to do great work?")
Markdown(res[0])

interesting, which you necessarily will if you're on the right path, the work will probably feel less burdensome than a lot of your peers'.The discoveries are out there, waiting to be made. Why not by you? Notes[1] I don't think you could give a precise definition of what counts as great work. Doing great work means doing something important so well that you expand people's ideas of what's possible. But there's no threshold for importance. It's a matter of degree, and often hard to judge at the

In [5]:
import fastrepl

# grab one from app.fastrepl.com
fastrepl.api_key = ""

In [6]:
generator = fastrepl.QuestionGenerator(source="http://paulgraham.com/greatwork.html")

In [7]:
ds = fastrepl.remote_runner(generator=generator).run()

Output()

Interrupted!
You can retrieve the job status with `run(job_id='fc-mhQTchhxonldFZtIxTlTDi')`

In [8]:
ds = fastrepl.remote_runner(generator=generator).run(job_id="fc-mhQTchhxonldFZtIxTlTDi")

Output()

In [9]:
ds

fastrepl.Dataset({
    features: ['question'],
    num_rows: 43
})

In [10]:
questions = ds["question"][:10]
questions[:1]

["How does the concept of morale affect one's ability to do great work?"]

In [13]:
ds = fastrepl.local_runner(fn=pg_bot.query).run(
    args_list=[(q,) for q in questions], output_feature="answer"
)

answers = ds["answer"]
answers[:1]

Output()

["The concept of morale affects one's ability to do great work by serving as the basis for everything when working on ambitious projects. It is important to nurture and protect morale like a living organism. Being an optimist and having a positive view of life can increase the likelihood of doing great work."]

In [14]:
ds = fastrepl.local_runner(fn=pg_bot.retrieve_from_database).run(
    args_list=[(q,) for q in questions], output_feature="contexts"
)

contexts = ds["contexts"]
contexts[:1]

Output()

[["have to become a manager, and managing well takes aptitude and interest like any other kind of work. If you don't have them, there is no middle path: you must either force yourself to learn management as a second language, or avoid such projects. [27] Husband your morale. It's the basis of everything when you're working on ambitious projects. You have to nurture and protect it like a living organism.Morale starts with your view of life. You're more likely to do great work if you're an optimist,"]]

In [15]:
ds = fastrepl.Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)
ds

fastrepl.Dataset({
    features: ['question', 'answer', 'contexts'],
    num_rows: 10
})

In [16]:
evaluator = fastrepl.RAGEvaluator(
    node=fastrepl.RAGAS(model="gpt-3.5-turbo", metric="AnswerRelevancy")
)

In [17]:
result = fastrepl.local_runner(evaluator=evaluator, dataset=ds).run(show_progress=False)
result

fastrepl.Dataset({
    features: ['question', 'answer', 'contexts', 'result'],
    num_rows: 10
})

In [18]:
result["result"]

[0.9626099323357186,
 0.9999995231647457,
 0.9896081791118906,
 0.999998666912428,
 0.9999989920279274,
 0.9863811768369456,
 0.8960008048863152,
 0.835167892418118,
 0.9771222554067247,
 0.9968260461237679]